<a href="https://colab.research.google.com/github/poudel-bibek/Intro-to-AI-Assignments/blob/main/A8_Task.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![A8](https://user-images.githubusercontent.com/96804013/153293943-c89fabd1-8c6d-471c-ac21-f3d40cf1fb3c.png)


# Assignment 8: Bandits (Task)
---

In this assignment, we will be solving the clssic Reinforcement Learning problem known as multi-armed bandits ([link](https://en.wikipedia.org/wiki/Multi-armed_bandit)).

<p align="center">
  <img src="https://user-images.githubusercontent.com/96804013/153735094-7c469a0b-2c05-4489-a436-eefb8128a379.png")
"/>
</p>

<p align="center">
  <em>Figure 1: The Multi-armed bandit problem</em>
</p>


__The problem:__ 
  - Imagine that you are in a casino with 4 different slot machines each known as `"one-armed bandit"` (as they're known for robbing people). Each bandit consists of a lever/ arm that can be pulled to receive a payout. 
  - Some slot machines payout more frequently than others i.e., each bandit has its own probability of success which remains unkown to us.
  - Our is to walk out of the casino with the most money. i.e., pull the bandit arm one by one to maximize the total reward collected.


<!-- __Solution:__
  - We will train an agent which we allow to choose actions, and each action has a reward that is returned according to a given, underlying probability distribution. -->

Start by importing the necessary packages: 

                    import os
                    import time
                    import random
                    import numpy as np
                    import matplotlib.pyplot as plt


Our problem formulation is episodic, i.e., we will conduct many episodes each with multiple timesteps.

At each timestep,

- We take an action on one slot machine i.e., pick which bandit arm to pull. The version of Multi-armed bandits that we implement is known as `Bernoulli Bandits`, based on the idea of `Bernoulli experiments`, where each bandit behaves like a random variable with binary outcomes i.e., every time we pull a bandit arm, it can have exactly two outcomes `"success"` or `"failure"`(corresponding to the reward of either a 0 or a 1). 
- Further, the probability of success remains the same every time the experiment is conducted. 


Lets use the following code to implement the reward behavior on a single timestep: 

                  def timestep(probabilities, arm_to_pull):
                    rand_num = random.random()
                    if rand_num <= probabilities[arm_to_pull]:
                      reward = 1
                    else: 
                      reward = 0
                    return reward 

Note that, in the timestep function above, to provide a reward on each timestep, the function has access to the `"true"` payout probabilities. 

---

Next, we will start by defininig an `Agent` that:

- Keeps a track of:
  1. Actions taken over an episode i.e, number of times each bandit arm was pulled over an episode. Remember that at each timestep, only a single arm can be pulled.
  
  2. `"Quality"` values of each action taken that estimates the expected reward that we can get on that action. A quality value indicates how good is it to take that particular action. At the start of an episode, quality values are initialized to zero and every timestep the quality values are updated.

- Further, the agent has the capabilities to:
  1. Take an action: the agent can choose which bandit arm to pull each timestep. To choose which arm to pull, it will use an `ϵ-greedy` (Epsilon greedy) action-selection strategy i.e., we balance exploration and exploitation such that the arm with highest known payout (based on maximum quality value) is chosen `1-ϵ` fraction of times and a randomly chosen arm is pulled `ϵ` fraction of times.
 
  2. Update quality value: The quality value of an action is updated using the reward received after taking that action and the fraction of times that particular action has been taken this episode so far. At each timestep, only a single quality value (of one action) is updated using the update rule given by:

  $$Quality~(a) ←  Quality~(a) + \frac{1.0}{\mathcal{N}_a}\cdot[Reward(a) - Quality~(a)],$$

where, $\mathcal{N}_a = $ Number of times this action has been taken so far in this episode. Note that, the agent has no idea what the `"true"` payout probability of each bandit arm is.
<br>

<!-- Self-notes: Bibek: This has one problem, if all of them have equal Q-values, the probability of sampling is not equal. 0 is always chosen in this case because I use np.argmax() and for multiple indices with same Q-values, it always returns a 0. The correct implementaion should be randomly choose all indices with equal probabilities if they have same Q values. But this does not affect the results too much because such occurences are extremely rare (<1%) -->

Use the following code to implement an agent: 

                    class Agent:
                      def __init__(self, num_actions):
                        self.num_actions = num_actions 
                        self.actions_track = np.zeros(num_actions, dtype=np.int) 
                        self.quality_values = np.zeros(num_actions, dtype=np.float)

                      def update_quality_value(self, action, reward):
                        self.actions_track[action] += 1
                        self.quality_values[action] += (1.0/self.actions_track[action]) * (reward - self.quality_values[action])
                        
                      def get_action(self, eps):
                        if np.random.random() < eps: # explore
                          return np.random.randint(self.num_actions)
                        else: 
                          return np.argmax(self.quality_values)


An episode starts by initializing a new agent after which we go over a number of timesteps where,

Each timestep in an episode is a `Bernoulli experiment` where: 
  - Ask the agent what action to take
  - Take a timestep with that action and collect reward 
  - Update quality values for the action taken

After the timestep ends, we collect actions and rewards over the episode 

---
##Exercise 1

__episode function:__
- Takes as input the `true` payout probabilities (for reward calulation) and number of time steps in an episode. 

- Ouputs the actions taken in each timestep over an episode as well as the rewards collected from each actions over an episode.

In the implementation of the episode function below, fill in your code:

                    def episode(probs, steps_per_episode):
                      agent = Agent(len(probs))
                      actions_per_episode= [] 
                      rewards_per_episode = []

                      for step in range(steps_per_episode):

                        action = ######## YOUR CODE HERE (1) #########
                        reward = ######## YOUR CODE HERE (2) #########
                        
                        agent.update_quality_value(action, reward)

                        actions_per_episode.append(action)
                        rewards_per_episode.append(reward)

                      return np.array(actions_per_episode), np.array(rewards_per_episode)

- Hints:
    1. you can use `agent.get_action()` to get an action
    2. reward is obtained each timestep by calling the `timestep` function defined above

Next, Use the following code to set some parameters and settings: 

                    probs = [0.10, 0.50, 0.60, 0.80] 
                    num_arms = len(probs)
                    max_episodes = 10000 

                    steps_per_episode = 500 
                    eps = 0.1 ##### CHANGE THIS VALUE ######

                    total_rewards = np.zeros((steps_per_episode,))  # reward history sum
                    total_actions = np.zeros((steps_per_episode, num_arms))  # action history sum

                    print(f"Rewards to collect: {total_rewards.shape}\nActions to collect: {total_actions.shape}")


- `total_rewards` keeps a track of rewards collected at each timestep over episodes.
- `total_actions` keeps a track of actions taken at each timestep over episodes.

Initialize `total_rewards` and `total_actions` keep a track of all rewards collected and actions taken each episode 

---
##Exercise 2

Re-run the entire Colab with with different values of `eps` below:
Values should be between 0 and 1. 

- `eps` = 0 means the no exploration i.e., always take the best known action 
- `eps` = 1 means full exploration i.e., always take a random action

What did you find?

- Each episode is run independent of other i.e., there is no transfer of `knowledge` (things that the agent learns over an episode/ quality values) are not transferred from one episode to next.

Now, use the code below to run all episodes and collect history of rewards and actions taken: 

                    for i in range(max_episodes):
                      actions_episode, rewards_episode = episode(probs, steps_per_episode)  
                      avg_reward_per_timestep = np.sum(rewards_episode) / steps_per_episode

                      if i  % 100 == 0:
                        print(f"Episode {i}: average reward per timestep = {avg_reward_per_timestep}")

                      total_rewards += rewards_episode
                      for j, a in enumerate(actions_episode):
                        total_actions[j][a] += 1


After performing one `Bernoulli Experiment` for each timestep in each episode the agent gets estimation of the unkown probabilities. 

Plot 1: 
- Average reward each timestep 

Plot 2: 
- Across 10,000 experiments, the rate of arm selection per timestep
- After 10,000 experiments, we are able to `"estimate"` the payout probabilities of each bandit.

Why are the `x-axes` in the plots are timesteps?
 
 <p align="center">
  <img src="https://user-images.githubusercontent.com/96804013/153914850-08bf6212-e837-42e9-9336-e61a3ffa4185.png")
"/>
</p>

<p align="center">
  <em>Figure 2: Average reward per timestep</em>
</p>

<p align="center">
  <img src="https://user-images.githubusercontent.com/96804013/153914853-79a265f5-d176-4f05-b4a1-ec61414d7e6a.png")
"/>
</p>

<p align="center">
  <em>Figure 3: Probability estimation (The arm probabilities may be different in your case, try to find the eps that gets this probabilities)</em>
</p>

Use the following code to plot:

                    # Plot reward results

                    average_reward_per_timestep = total_rewards / max_episodes

                    fig, ax = plt.subplots(figsize=(12, 5), dpi = 80)
                    plt.grid()
                    ax.plot(average_reward_per_timestep, '.')
                    ax.set_xlabel("timestep", fontsize=16)
                    ax.set_ylabel("Average Reward per timestep", fontsize=16)
                    ax.set_ylim([0,1])
                    ax.set_xlim([1, steps_per_episode])

                    # Plot action results

                    fig, ax = plt.subplots(figsize=(12, 5), dpi = 80)
                    plt.grid()
                    for i in range(num_arms):
                      arm_pull_average = 100 * total_actions[:,i] / max_episodes
                      steps = list(np.array(range(steps_per_episode))+1)
                      ax.plot(steps, arm_pull_average, '-', linewidth = 2, label =f"Arm{i}: {arm_pull_average[-1]}%")

                    ax.legend(fontsize=14)
                    ax.set_xlabel("timestep", fontsize=16)
                    ax.set_ylabel("Average (%) arm choice over timesteps", fontsize=16)
                    ax.set_xlim([0, steps_per_episode])
                    ax.set_ylim([0, 100])

